In [1]:
from pynq import allocate
from pynq import Overlay

import numpy as np
import time
import cmath
import os

import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.display import Image
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [2]:
# Define CSS for background colour only
custom_style = """
    <style>
        body {
            background-color: #CCB7E5 !important; /* Set background color to pale purple */
        }
        .custom-button {
            font-size: 17px !important;  /* Larger button text */
            padding: 25px 60px !important;  /* Larger button padding */
            background-color: #FF69B4 !important; /* Hot pink buttons */
            color: white !important; /* White text */
            font-weight: bold !important; /* Bold text */
            border: none !important;
            font-family: 'Press Start 2P', cursive;
            width: 250px; /* Set fixed width for the buttons */
        }
    </style>
"""

# Apply the custom style to the notebook
display(widgets.HTML(value=custom_style))

HTML(value="\n    <style>\n        body {\n            background-color: #CCB7E5 !important; /* Set background…

In [3]:
def generate_random_qubit():
    # Generate random real and imaginary parts for alpha and beta
    a_real = np.random.uniform(-1, 1)
    a_imag = np.random.uniform(-1, 1)
    b_real = np.random.uniform(-1, 1)
    b_imag = np.random.uniform(-1, 1)
    
    # Create complex numbers for alpha and beta
    alpha = complex(a_real, a_imag)
    beta = complex(b_real, b_imag)
    
    # Normalise to ensure |alpha|^2 + |beta|^2 = 1
    norm = np.sqrt(abs(alpha)**2 + abs(beta)**2)
    alpha /= norm
    beta /= norm
    
    # Output real and imaginary parts separately
    return alpha.real, alpha.imag, beta.real, beta.imag

def generate_control_qubit():
    # Set alpha to 0 and beta to 1 for the |1> state
    alpha_real = 0.0
    alpha_imag = 0.0
    beta_real = 1.0
    beta_imag = 0.0
    
    # Return the components of the qubit
    return alpha_real, alpha_imag, beta_real, beta_imag


In [4]:
# Function to convert qubit to Bloch sphere coordinates
def qubit_to_bloch(alpha_real, alpha_imag, beta_real, beta_imag):
    alpha = complex(alpha_real, alpha_imag)
    beta = complex(beta_real, beta_imag)

    # Normalize qubit state
    norm = np.sqrt(abs(alpha)**2 + abs(beta)**2)
    alpha /= norm
    beta /= norm

    theta = 2 * np.arccos(abs(alpha))
    phi = np.angle(beta) - np.angle(alpha)

    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    
    return x, y, z, alpha, beta  # Return alpha and beta for later use in the plot

# Function to plot Bloch sphere and display qubit info
def plot_bloch_sphere(x=0, y=0, z=0, alpha=None, beta=None):
    fig = go.Figure()

    # Draw Bloch sphere surface
    u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
    xs = np.cos(u) * np.sin(v)
    ys = np.sin(u) * np.sin(v)
    zs = np.cos(v)

    fig.add_trace(go.Surface(x=xs, y=ys, z=zs, opacity=0.35, colorscale=[[0, 'lavenderblush'], [1, 'thistle']]))

    # Plot qubit state vector
    fig.add_trace(go.Scatter3d(x=[x], y=[y], z=[z], mode='markers',
                                marker=dict(color='deeppink', size=6), name='Qubit'))
    fig.add_trace(go.Scatter3d(x=[0, x], y=[0, y], z=[0, z], mode='lines',
                                line=dict(color='deeppink', width=4), name='Qubit Vector'))

    # Add |0> and |1> labels
    fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[1], mode='markers+text',
                                marker=dict(color='darkviolet', size=8), text='|0⟩',
                                textposition='bottom center', textfont=dict(size=16, color='black')))
    fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[-1], mode='markers+text',
                                marker=dict(color='mediumpurple', size=8), text='|1⟩',
                                textposition='top center', textfont=dict(size=16, color='black')))

    # Display qubit info (alpha and beta)
    if alpha is not None and beta is not None:
        qubit_info = f"|ψ⟩ = {alpha:.2f}|0⟩ + {beta:.2f}|1⟩"
        fig.add_trace(go.Scatter3d(
            x=[0], y=[0], z=[1.3], mode='text',
            text=[qubit_info], textposition='top center',
            textfont=dict(size=16, color='black')
        ))

    # Add label for the qubit state at the end of the pink arrow
    fig.add_trace(go.Scatter3d(
        x=[x], y=[y], z=[z], mode='text',
        text=["|ψ⟩"], textposition='top right',
        textfont=dict(size=16, color='deeppink')
    ))

    fig.update_layout(
        title=None,  # Remove title
        plot_bgcolor='lavenderblush', paper_bgcolor='#CCB7E5',
        font=dict(color='black'), height=700, width=1300,
        scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
    )

    return fig

# Initializing the Bloch sphere without a qubit
def initial_bloch_sphere():
    return plot_bloch_sphere()

# Generate a random qubit and visualize it
def visualize_random_qubit():
    alpha_r, alpha_i, beta_r, beta_i = generate_random_qubit()
    x, y, z, alpha, beta = qubit_to_bloch(alpha_r, alpha_i, beta_r, beta_i)
    return plot_bloch_sphere(x, y, z, alpha, beta)



In [5]:
# Used in gates: X, Y, Z, Hadamard Gates (1 input gates) 
# Converts qubit data into 32 bit unsigned and packs ready to be streamed via axi. 
###########################################################################################
def convert_to_unsigned_fixed_point(alpha_real, alpha_imag, beta_real, beta_imag):
    # Convert each component to 32-bit signed fixed-point representation
    fixed_alpha_real = np.int32(alpha_real * (2**31))
    fixed_alpha_imag = np.int32(alpha_imag * (2**31))
    fixed_beta_real = np.int32(beta_real * (2**31))
    fixed_beta_imag = np.int32(beta_imag * (2**31))
    
    # Convert to unsigned by adding 2^32 to negative values
    unsigned_alpha_real = np.uint32(fixed_alpha_real if fixed_alpha_real >= 0 else fixed_alpha_real + 2**32)
    unsigned_alpha_imag = np.uint32(fixed_alpha_imag if fixed_alpha_imag >= 0 else fixed_alpha_imag + 2**32)
    unsigned_beta_real = np.uint32(fixed_beta_real if fixed_beta_real >= 0 else fixed_beta_real + 2**32)
    unsigned_beta_imag = np.uint32(fixed_beta_imag if fixed_beta_imag >= 0 else fixed_beta_imag + 2**32)
    
    # Pack into a 1D 128-bit array (4 x 32-bit unsigned integers)
    unsigned_fixed_point_array = np.array([unsigned_alpha_real, unsigned_alpha_imag, unsigned_beta_real, unsigned_beta_imag], dtype=np.uint32)
    return unsigned_fixed_point_array


In [6]:
# Used in gates: SWAP (2 input gates)
# Converts qubit data into 16 bit unsigned and packs ready to be streamed via axi. 
############################################################################################
def convert_to_unsigned_fixed_point_16bit(q1_alpha_real, q1_alpha_imag, q1_beta_real, q1_beta_imag,
                                           q2_alpha_real, q2_alpha_imag, q2_beta_real, q2_beta_imag):
    # Convert each component to 16-bit signed fixed-point representation
    fixed_q1_alpha_real = np.int16(q1_alpha_real * (2**15))
    fixed_q1_alpha_imag = np.int16(q1_alpha_imag * (2**15))
    fixed_q1_beta_real = np.int16(q1_beta_real * (2**15))
    fixed_q1_beta_imag = np.int16(q1_beta_imag * (2**15))

    fixed_q2_alpha_real = np.int16(q2_alpha_real * (2**15))
    fixed_q2_alpha_imag = np.int16(q2_alpha_imag * (2**15))
    fixed_q2_beta_real = np.int16(q2_beta_real * (2**15))
    fixed_q2_beta_imag = np.int16(q2_beta_imag * (2**15))

    # Convert to unsigned by adding 2^16 to negative values
    unsigned_q1_alpha_real = np.uint16(fixed_q1_alpha_real if fixed_q1_alpha_real >= 0 else fixed_q1_alpha_real + 2**16)
    unsigned_q1_alpha_imag = np.uint16(fixed_q1_alpha_imag if fixed_q1_alpha_imag >= 0 else fixed_q1_alpha_imag + 2**16)
    unsigned_q1_beta_real = np.uint16(fixed_q1_beta_real if fixed_q1_beta_real >= 0 else fixed_q1_beta_real + 2**16)
    unsigned_q1_beta_imag = np.uint16(fixed_q1_beta_imag if fixed_q1_beta_imag >= 0 else fixed_q1_beta_imag + 2**16)

    unsigned_q2_alpha_real = np.uint16(fixed_q2_alpha_real if fixed_q2_alpha_real >= 0 else fixed_q2_alpha_real + 2**16)
    unsigned_q2_alpha_imag = np.uint16(fixed_q2_alpha_imag if fixed_q2_alpha_imag >= 0 else fixed_q2_alpha_imag + 2**16)
    unsigned_q2_beta_real = np.uint16(fixed_q2_beta_real if fixed_q2_beta_real >= 0 else fixed_q2_beta_real + 2**16)
    unsigned_q2_beta_imag = np.uint16(fixed_q2_beta_imag if fixed_q2_beta_imag >= 0 else fixed_q2_beta_imag + 2**16)

    # Pack into a 1D array of 16-bit unsigned integers
    unsigned_fixed_point_array = np.array([unsigned_q1_alpha_real, unsigned_q1_alpha_imag,
                                           unsigned_q1_beta_real, unsigned_q1_beta_imag,
                                           unsigned_q2_alpha_real, unsigned_q2_alpha_imag,
                                           unsigned_q2_beta_real, unsigned_q2_beta_imag], dtype=np.uint16)

    return unsigned_fixed_point_array


In [7]:
# Used in gates: S and T (Requires control qubit in |1> state)
# Converts qubit data into 16 bit unsigned and packs ready to be streamed via axi.
############################################################################################
def convert_to_unsigned_fixed_point_16bit_control(q1_alpha_real, q1_alpha_imag, q1_beta_real, q1_beta_imag,
                                                  control_alpha_real, control_alpha_imag, control_beta_real, control_beta_imag):
    # Convert both Qubit 1 and the control qubit to 16-bit signed fixed-point representation
    fixed_q1_alpha_real = np.int16(q1_alpha_real * (2**15))
    fixed_q1_alpha_imag = np.int16(q1_alpha_imag * (2**15))
    fixed_q1_beta_real = np.int16(q1_beta_real * (2**15))
    fixed_q1_beta_imag = np.int16(q1_beta_imag * (2**15))

    fixed_control_alpha_real = np.int16(control_alpha_real * (2**15))
    fixed_control_alpha_imag = np.int16(control_alpha_imag * (2**15))
    fixed_control_beta_real = np.int16(control_beta_real * (2**15))
    fixed_control_beta_imag = np.int16(control_beta_imag * (2**15))

    # Convert to unsigned by adding 2^16 to negative values
    unsigned_q1_alpha_real = np.uint16(fixed_q1_alpha_real if fixed_q1_alpha_real >= 0 else fixed_q1_alpha_real + 2**16)
    unsigned_q1_alpha_imag = np.uint16(fixed_q1_alpha_imag if fixed_q1_alpha_imag >= 0 else fixed_q1_alpha_imag + 2**16)
    unsigned_q1_beta_real = np.uint16(fixed_q1_beta_real if fixed_q1_beta_real >= 0 else fixed_q1_beta_real + 2**16)
    unsigned_q1_beta_imag = np.uint16(fixed_q1_beta_imag if fixed_q1_beta_imag >= 0 else fixed_q1_beta_imag + 2**16)

    unsigned_control_alpha_real = np.uint16(fixed_control_alpha_real if fixed_control_alpha_real >= 0 else fixed_control_alpha_real + 2**16)
    unsigned_control_alpha_imag = np.uint16(fixed_control_alpha_imag if fixed_control_alpha_imag >= 0 else fixed_control_alpha_imag + 2**16)
    unsigned_control_beta_real = np.uint16(fixed_control_beta_real if fixed_control_beta_real >= 0 else fixed_control_beta_real + 2**16)
    unsigned_control_beta_imag = np.uint16(fixed_control_beta_imag if fixed_control_beta_imag >= 0 else fixed_control_beta_imag + 2**16)

    # Pack into a 1D array of 16-bit unsigned integers (for Qubit 1 and control qubit)
    unsigned_control_fixed_point_array = np.array([ unsigned_control_alpha_real, unsigned_control_alpha_imag,
                                                   unsigned_control_beta_real, unsigned_control_beta_imag, unsigned_q1_alpha_real, unsigned_q1_alpha_imag,
                                                   unsigned_q1_beta_real, unsigned_q1_beta_imag], dtype=np.uint16)

    return unsigned_control_fixed_point_array


In [8]:
def apply_gate(gate_type, unsigned_fixed_point_data):
    """
    Apply quantum gate based on the selected gate type and manage DMA transfers accordingly.
    """
    global elapsed_time
    # Dynamically allocate input and output buffers based on the gate type
    if gate_type == ["S", "T", "Swap"]:
        # For Swap gate, two qubits are needed, so allocate based on the second data
        input_buffer = allocate(shape=(unsigned_fixed_point_data.size,), dtype=np.uint16)
        output_buffer = allocate(shape=(unsigned_fixed_point_data.size,), dtype=np.uint16)
        np.copyto(input_buffer, unsigned_fixed_point_data)

    else:
        # For all other gates like X, Y, Z, Hadamard, only one qubit is involved
        input_buffer = allocate(shape=(unsigned_fixed_point_data.size,), dtype=np.uint32)
        output_buffer = allocate(shape=(unsigned_fixed_point_data.size,), dtype=np.uint32)
        np.copyto(input_buffer, unsigned_fixed_point_data)

    # Perform the data transfer to PL and back to PS
    start = time.time()

    # Transfer Data to and from the PL
    dma.recvchannel.transfer(output_buffer)
    dma.sendchannel.transfer(input_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()

    elapsed_time = time.time() - start

    return output_buffer 


In [9]:
# This function loads the appropriate bitstream for the selected gate
def load_gate_bitstream(gate_choice):
    bitstream_mapping = {
        "X": "XGate.bit",
        "Y": "YGate.bit",
        "Z": "ZGate.bit",
        "Hadamard": "HadamardGate.bit",
        "S": "SGate.bit",
        "T": "TGate.bit",
        "Swap": "SWAPGate.bit"
    }

    if gate_choice in bitstream_mapping:
        # Load the corresponding bitstream based on the user's choice
        ol = Overlay(bitstream_mapping[gate_choice])
        return ol
    else:
        print("Invalid gate selected.")
        return None

In [10]:
# Function to convert 32 bit unsigned output to floating point
############################################################################################
def convert_from_unsigned_fixed_point(output_buffer):
    # Scaling factor for fixed-point conversion
    scaling_factor = 1 / (2**31)
    
    # Convert unsigned 32-bit values back to signed 32-bit values
    signed_alpha_real = np.int32(output_buffer[0] - 2**32 if output_buffer[0] >= 2**31 else output_buffer[0])
    signed_alpha_imag = np.int32(output_buffer[1] - 2**32 if output_buffer[1] >= 2**31 else output_buffer[1])
    signed_beta_real = np.int32(output_buffer[2] - 2**32 if output_buffer[2] >= 2**31 else output_buffer[2])
    signed_beta_imag = np.int32(output_buffer[3] - 2**32 if output_buffer[3] >= 2**31 else output_buffer[3])

    # Convert to floating-point by applying the scaling factor
    alpha_real = signed_alpha_real * scaling_factor
    alpha_imag = signed_alpha_imag * scaling_factor
    beta_real = signed_beta_real * scaling_factor
    beta_imag = signed_beta_imag * scaling_factor

    return alpha_real, alpha_imag, beta_real, beta_imag

In [11]:
# Function to convert 16 bit unsigned output to floating point
############################################################################################
def convert_from_unsigned_fixed_point_16bit(output_buffer):
    # Scaling factor for fixed-point conversion (using 16 bits)
    scaling_factor = 1 / (2**15)

    def to_signed(value):
        """Convert unsigned to signed 16-bit value."""
        # If the value is 32768 (middle of the range), treat it as positive, not negative
        if value == 32768:
            return 32768
        elif value >= 2**15:
            return value - 2**16  # Convert to signed value
        return value
    
    # Convert the unsigned values to signed values
    signed_q1_alpha_real = to_signed(output_buffer[0])
    signed_q1_alpha_imag = to_signed(output_buffer[1])
    signed_q1_beta_real = to_signed(output_buffer[2])
    signed_q1_beta_imag = to_signed(output_buffer[3])

    signed_q2_alpha_real = to_signed(output_buffer[4])
    signed_q2_alpha_imag = to_signed(output_buffer[5])
    signed_q2_beta_real = to_signed(output_buffer[6])
    signed_q2_beta_imag = to_signed(output_buffer[7])

    # Convert to floating-point by applying the scaling factor
    q1_alpha_real = signed_q1_alpha_real * scaling_factor
    q1_alpha_imag = signed_q1_alpha_imag * scaling_factor
    q1_beta_real = signed_q1_beta_real * scaling_factor
    q1_beta_imag = signed_q1_beta_imag * scaling_factor

    q2_alpha_real = signed_q2_alpha_real * scaling_factor
    q2_alpha_imag = signed_q2_alpha_imag * scaling_factor
    q2_beta_real = signed_q2_beta_real * scaling_factor
    q2_beta_imag = signed_q2_beta_imag * scaling_factor

    
    return q1_alpha_real, q1_alpha_imag, q1_beta_real, q1_beta_imag, q2_alpha_real, q2_alpha_imag, q2_beta_real, q2_beta_imag


In [12]:
# Function to convert 32 bit unsigned output and reverse scaling of 2^4 to floating point
# Used in Hadamard gate.
############################################################################################
def convert_and_reverse_scaling(output_buffer, scale_factor=16):
    scaling_factor_fixed_point = 1 / (2**31)  # For converting back from signed fixed-point 32-bit
    reverse_scaling = 1 / scale_factor        # Reverse the scaling applied in Model Composer

    # Convert from unsigned back to signed 32-bit fixed-point
    signed_alpha_real = np.int32(output_buffer[0] - 2**32 if output_buffer[0] >= 2**31 else output_buffer[0])
    signed_alpha_imag = np.int32(output_buffer[1] - 2**32 if output_buffer[1] >= 2**31 else output_buffer[1])
    signed_beta_real = np.int32(output_buffer[2] - 2**32 if output_buffer[2] >= 2**31 else output_buffer[2])
    signed_beta_imag = np.int32(output_buffer[3] - 2**32 if output_buffer[3] >= 2**31 else output_buffer[3])

    # Apply both scaling corrections
    alpha_real = signed_alpha_real * scaling_factor_fixed_point * reverse_scaling
    alpha_imag = signed_alpha_imag * scaling_factor_fixed_point * reverse_scaling
    beta_real = signed_beta_real * scaling_factor_fixed_point * reverse_scaling
    beta_imag = signed_beta_imag * scaling_factor_fixed_point * reverse_scaling

    return alpha_real, alpha_imag, beta_real, beta_imag


In [13]:
def display_gate_selection():
    # Initialise the output widget for displaying results
    output = widgets.Output()

    # Add the title for the introduction to quantum gates
    gate_title = widgets.HTML(
        value="<h2 style='color:white; font-size:40px;'>Introduction to Quantum Gates</h2>"
    )

    # Add text explaining what a quantum gate is
    gate_explanation = widgets.HTML(
        value=""" 
        <div style="background-color:#f3e5f5; padding:20px; border-radius:10px; 
                    width: 70%; margin-left: 0; margin-right: 5%; box-shadow: 2px 2px 10px rgba(0,0,0,0.2);">
            <h3 style="color:#6a4c9c; font-size:24px;">What is a Quantum Gate?</h3>
            <p style="color:#6a4c9c; font-size:18px;">
                A quantum gate is a basic operation that manipulates a qubit’s state. Similar to classical logic gates, 
                quantum gates perform operations on qubits, but unlike classical gates, quantum gates are reversible 
                and can operate on the complex probabilities of qubit states. Quantum gates are the building blocks 
                for quantum algorithms, such as the Quantum Fourier Transform and Shor’s algorithm.
            </p>
        </div>
        """
    )

    # Create a prompt for the user
    gate_prompt = widgets.HTML(
        value="<h3 style='text-align:left; color:purple;'>Select a Quantum Gate to Simulate</h3>"
    )

    # Create the dropdown for quantum gate selection
    gate_dropdown = widgets.Dropdown(
        options=['X', 'Y', 'Z', 'Hadamard', 'S', 'T', 'Swap'],
        value='X',  # Default value
        description='Gate:',
        disabled=False,
        style={'description_width': 'initial', 'font_size': '20px'}  
    )

    # Create a button to proceed with the selected gate
    simulate_button = widgets.Button(
        description="Simulate Gate",
        layout=widgets.Layout(width='250px', height='80px'),
        style={'button_color': '#FF69B4', 'color': 'white'}
    )

    simulate_button.add_class('custom-button')

    # Create a VBox for the text section (title and explanation)
    text_section = widgets.VBox([gate_title, gate_explanation], layout=widgets.Layout(align_items='flex-start', width="100%"))

    # Create a VBox for dropdown and button
    controls_section = widgets.VBox([gate_prompt, gate_dropdown, simulate_button], layout=widgets.Layout(align_items='flex-start', width="100%"))

    # Stack text_section, controls_section, and the output widget vertically
    final_layout = widgets.VBox([text_section, controls_section, output], layout=widgets.Layout(align_items='flex-start'))

    # Display the content
    display(final_layout)
    
    def on_gate_button_clicked(b):
        global dma
        with output:
            output.clear_output(wait=True)

            # Get selected gate from the dropdown
            selected_gate = gate_dropdown.value

            # Load the bitstream for the selected gate
            ol = load_gate_bitstream(selected_gate)
            if ol is None:
                print(f"Error loading the bitstream for {selected_gate}.")
                return  # Exit early if loading fails

            # Get the AXI DMA from the loaded bitstream
            dma = ol.axi_dma

            # Display gate information
            display(widgets.HTML(f"<h3 style='font-size:25px; color:purple;'>Simulating gate: {selected_gate}</h3>"))

            if selected_gate == "Hadamard":
                # Explanation for Hadamard gate
                display(widgets.HTML("<p style='font-size:25px; color:purple;'>The Hadamard gate (H) creates a superposition state from a basis state.</p>"))

                # Generate and apply random qubit data
                alpha_real, alpha_imag, beta_real, beta_imag = generate_random_qubit()
                unsigned_fixed_point_data = convert_to_unsigned_fixed_point(alpha_real, alpha_imag, beta_real, beta_imag)
                output_buffer = apply_gate("Hadamard", unsigned_fixed_point_data)

                # Convert output buffer data using reverse scaling for Hadamard
                alpha_realO, alpha_imagO, beta_realO, beta_imagO = convert_and_reverse_scaling(output_buffer)

                # Display initial qubit state
                display(widgets.HTML("<p style='font-size:22px; color:purple;'>Initial Qubit State</p>"))
                x, y, z, alpha, beta = qubit_to_bloch(alpha_real, alpha_imag, beta_real, beta_imag)
                fig = plot_bloch_sphere(x, y, z, alpha, beta)
                display(fig)

                # Display output qubit state after applying Hadamard gate
                display(widgets.HTML("<p style='font-size:22px; color:purple;'>Qubit state after applying Hadamard gate:</p>"))
                display(widgets.HTML(f"<p style='font-size:22px; color:purple;'>Time taken to emulate: {elapsed_time:.6f} seconds</p>"))
                x_out, y_out, z_out, alpha_out, beta_out = qubit_to_bloch(alpha_realO, alpha_imagO, beta_realO, beta_imagO)
                fig_out = plot_bloch_sphere(x_out, y_out, z_out, alpha_out, beta_out)
                display(fig_out)
                
            elif selected_gate == "X":
                # Explanation for X gate
                gate_name = selected_gate
                display(widgets.HTML(f"<p style='font-size:25px; color:purple;'>The X gate (also called the NOT gate) flips the state of a qubit.</p>"))

                # Generate and apply random qubit data
                alpha_real, alpha_imag, beta_real, beta_imag = generate_random_qubit()
                unsigned_fixed_point_data = convert_to_unsigned_fixed_point(alpha_real, alpha_imag, beta_real, beta_imag)
                output_buffer = apply_gate(gate_name, unsigned_fixed_point_data)

                # Convert output buffer data (32-bit conversion)
                alpha_realO, alpha_imagO, beta_realO, beta_imagO = convert_from_unsigned_fixed_point(output_buffer)

                # Display initial qubit state
                display(widgets.HTML("<p style='font-size:22px; color:purple;'>Initial Qubit State</p>"))
                x, y, z, alpha, beta = qubit_to_bloch(alpha_real, alpha_imag, beta_real, beta_imag)
                fig = plot_bloch_sphere(x, y, z, alpha, beta)
                display(fig)

                # Display output qubit state after applying the gate
                display(widgets.HTML(f"<p style='font-size:22px; color:purple;'>Qubit state after applying {gate_name} gate:</p>"))
                display(widgets.HTML(f"<p style='font-size:22px; color:purple;'>Time taken to emulate: {elapsed_time:.6f} seconds</p>"))
                x_out, y_out, z_out, alpha_out, beta_out = qubit_to_bloch(alpha_realO, alpha_imagO, beta_realO, beta_imagO)
                fig_out = plot_bloch_sphere(x_out, y_out, z_out, alpha_out, beta_out)
                display(fig_out)
                
            elif selected_gate == "Y":
                # Explanation for Y gate
                gate_name = selected_gate
                display(widgets.HTML(f"<p style='font-size:25px; color:purple;'>The Y gate performs a rotation around the Y-axis of the Bloch sphere.</p>"))

                # Generate and apply random qubit data
                alpha_real, alpha_imag, beta_real, beta_imag = generate_random_qubit()
                unsigned_fixed_point_data = convert_to_unsigned_fixed_point(alpha_real, alpha_imag, beta_real, beta_imag)
                output_buffer = apply_gate(gate_name, unsigned_fixed_point_data)

                # Convert output buffer data (32-bit conversion)
                alpha_realO, alpha_imagO, beta_realO, beta_imagO = convert_from_unsigned_fixed_point(output_buffer)

                # Display initial qubit state
                display(widgets.HTML("<p style='font-size:22px; color:purple;'>Initial Qubit State</p>"))
                x, y, z, alpha, beta = qubit_to_bloch(alpha_real, alpha_imag, beta_real, beta_imag)
                fig = plot_bloch_sphere(x, y, z, alpha, beta)
                display(fig)

                # Display output qubit state after applying the gate
                display(widgets.HTML(f"<p style='font-size:22px; color:purple;'>Qubit state after applying {gate_name} gate:</p>"))
                display(widgets.HTML(f"<p style='font-size:22px; color:purple;'>Time taken to emulate: {elapsed_time:.6f} seconds</p>"))
                x_out, y_out, z_out, alpha_out, beta_out = qubit_to_bloch(alpha_realO, alpha_imagO, beta_realO, beta_imagO)
                fig_out = plot_bloch_sphere(x_out, y_out, z_out, alpha_out, beta_out)
                display(fig_out)
                
            elif selected_gate == "Z":
                # Explanation for Z gate
                gate_name = selected_gate
                display(widgets.HTML(f"<p style='font-size:25px; color:purple;'>The Z gate applies a phase flip to the qubit.</p>"))

                # Generate and apply random qubit data
                alpha_real, alpha_imag, beta_real, beta_imag = generate_random_qubit()
                unsigned_fixed_point_data = convert_to_unsigned_fixed_point(alpha_real, alpha_imag, beta_real, beta_imag)
                output_buffer = apply_gate(gate_name, unsigned_fixed_point_data)

                # Convert output buffer data (32-bit conversion)
                alpha_realO, alpha_imagO, beta_realO, beta_imagO = convert_from_unsigned_fixed_point(output_buffer)

                # Display initial qubit state
                display(widgets.HTML("<p style='font-size:22px; color:purple;'>Initial Qubit State</p>"))
                x, y, z, alpha, beta = qubit_to_bloch(alpha_real, alpha_imag, beta_real, beta_imag)
                fig = plot_bloch_sphere(x, y, z, alpha, beta)
                display(fig)

                # Display output qubit state after applying the gate
                display(widgets.HTML(f"<p style='font-size:22px; color:purple;'>Qubit state after applying {gate_name} gate:</p>"))
                display(widgets.HTML(f"<p style='font-size:22px; color:purple;'>Time taken to emulate: {elapsed_time:.6f} seconds</p>"))
                x_out, y_out, z_out, alpha_out, beta_out = qubit_to_bloch(alpha_realO, alpha_imagO, beta_realO, beta_imagO)
                fig_out = plot_bloch_sphere(x_out, y_out, z_out, alpha_out, beta_out)
                display(fig_out)


            elif selected_gate == "Swap":
                # Explanation for Swap gate
                display(widgets.HTML("<p style='font-size:25px; color:purple;'>The Swap gate exchanges the states of two qubits.</p>"))

                # Generate and apply random qubit data for two qubits
                alpha_real, alpha_imag, beta_real, beta_imag = generate_random_qubit()
                alpha_real2, alpha_imag2, beta_real2, beta_imag2 = generate_random_qubit()
                unsigned_fixed_point_data = convert_to_unsigned_fixed_point_16bit(alpha_real, alpha_imag, beta_real, beta_imag,
                                                                                   alpha_real2, alpha_imag2, beta_real2, beta_imag2)
                output_buffer = apply_gate("Swap", unsigned_fixed_point_data)

                # Convert output buffer data using 16-bit conversion
                q1_alpha_realO, q1_alpha_imagO, q1_beta_realO, q1_beta_imagO, q2_alpha_realO, q2_alpha_imagO, q2_beta_realO, q2_beta_imagO = convert_from_unsigned_fixed_point_16bit(output_buffer)

                # Display initial qubit states
                display(widgets.HTML("<p style='font-size:22px; color:purple;'>Initial Qubit States</p>"))
                x1, y1, z1, alpha1, beta1 = qubit_to_bloch(alpha_real, alpha_imag, beta_real, beta_imag)
                x2, y2, z2, alpha2, beta2 = qubit_to_bloch(alpha_real2, alpha_imag2, beta_real2, beta_imag2)
                fig1 = plot_bloch_sphere(x1, y1, z1, alpha1, beta1)
                fig2 = plot_bloch_sphere(x2, y2, z2, alpha2, beta2)
                display(fig1)
                display(fig2)

                # Display output qubit states after swap gate
                display(widgets.HTML("<p style='font-size:22px; color:purple;'>Qubit states after applying Swap gate:</p>"))
                display(widgets.HTML(f"<p style='font-size:22px; color:purple;'>Time taken to emulate: {elapsed_time:.6f} seconds</p>"))
                x_out, y_out, z_out, alpha_out, beta_out = qubit_to_bloch(q1_alpha_realO, q1_alpha_imagO, q1_beta_realO, q1_beta_imagO)
                x_out2, y_out2, z_out2, alpha_out2, beta_out2 = qubit_to_bloch(q2_alpha_realO, q2_alpha_imagO, q2_beta_realO, q2_beta_imagO)
                fig_out = plot_bloch_sphere(x_out, y_out, z_out, alpha_out, beta_out)
                fig_out2 = plot_bloch_sphere(x_out2, y_out2, z_out2, alpha_out2, beta_out2)
                display(fig_out)
                display(fig_out2)
                
            elif selected_gate in ["S", "T"]:
                # Explanation for S and T gates
                gate_name = "S" if selected_gate == "S" else "T"
                display(widgets.HTML(f"<p style='font-size:25px; color:purple;'>The {gate_name} gate applies a phase shift on the qubit.</p>"))

                # Generate and apply random qubit data
                alpha_real, alpha_imag, beta_real, beta_imag = generate_random_qubit()
                control_alpha_real, control_alpha_imag, control_beta_real, control_beta_imag = generate_control_qubit()
                unsigned_fixed_point_data = convert_to_unsigned_fixed_point_16bit_control(alpha_real, alpha_imag, beta_real, beta_imag, control_alpha_real, control_alpha_imag, control_beta_real, control_beta_imag)
                output_buffer = apply_gate(gate_name, unsigned_fixed_point_data)

                # Convert output buffer data using 16-bit conversion
                control_alpha_realO, control_alpha_imagO, control_beta_realO, control_beta_imagO, q1_alpha_realO, q1_alpha_imagO, q1_beta_realO, q1_beta_imagO = convert_from_unsigned_fixed_point_16bit(output_buffer)

                # Display initial qubit state
                display(widgets.HTML("<p style='font-size:22px; color:purple;'>Initial Qubit State + Control Qubit</p>"))
                x, y, z, alpha, beta = qubit_to_bloch(alpha_real, alpha_imag, beta_real, beta_imag)
                x_c, y_c, z_c, alpha_c, beta_c = qubit_to_bloch(control_alpha_real, control_alpha_imag, control_beta_real, control_beta_imag)
                fig = plot_bloch_sphere(x, y, z, alpha, beta)
                fig2 = plot_bloch_sphere(x_c, y_c, z_c, alpha_c, beta_c)
                display(fig)
                display(fig2)

                display(widgets.HTML("<p style='font-size:22px; color:purple;'>Qubit state after applying phase shift gate:</p>"))
                display(widgets.HTML(f"<p style='font-size:22px; color:purple;'>Time taken to emulate: {elapsed_time:.6f} seconds</p>"))
                x_out, y_out, z_out, alpha_out, beta_out = qubit_to_bloch(q1_alpha_realO, q1_alpha_imagO, q1_beta_realO, q1_beta_imagO)
                x_out_c, y_out_c, z_out_c, alpha_out_c, beta_out_c = qubit_to_bloch(control_alpha_realO, control_alpha_imagO, control_beta_realO, control_beta_imagO)
                fig_out = plot_bloch_sphere(x_out, y_out, z_out, alpha_out, beta_out)
                fig_out2 = plot_bloch_sphere(x_out_c, y_out_c, z_out_c, alpha_out_c, beta_out_c)
                display(fig_out)
                display(fig_out2)


    # Link the button to the handler
    simulate_button.on_click(on_gate_button_clicked)

In [14]:
def write_qft_to_axi(overlay, state):
    """
    Write the quantum state to the AXI stream.
    """
    qubitInput = overlay.axi_stream_template_0
    
    # Ensure state is being written correctly   
    q0 = np.uint32(state & 1)
    q1 = np.uint32((state >> 1) & 1)
    q2 = np.uint32((state >> 2) & 1)
    
    # Write each bit to its corresponding gateway
    qubitInput.write(0x00, int(q0))
    qubitInput.write(0x04, int(q1))
    qubitInput.write(0x08, int(q2))

In [15]:
def read_qft_from_axi(overlay):
    """
    Read the quantum state from the AXI stream.
    """
    qubitInput = overlay.axi_stream_template_0
    
    # Read the state of each qubit from the AXI stream
    q0 = qubitInput.read(0x0C)  # Read the state of q0 from the corresponding address
    q1 = qubitInput.read(0x10)  # Read the state of q1 from the corresponding address
    q2 = qubitInput.read(0x14)  # Read the state of q2 from the corresponding address
    
    # Return the values as a tuple (q2, q1, q0)
    return q2, q1, q0

In [16]:
# Function to compute the tensor product of three quantum states
def tensor_product_three(state1, state2, state3):
    return np.kron(np.kron(state1, state2), state3)

In [17]:
# Function to compute the amplitude of the quantum state
def compute_amplitude(qubit_states):
    """
    Compute the tensor product of the 3 qubit states and normalize the amplitude.
    """
    total_state = tensor_product_three(qubit_states[0], qubit_states[1], qubit_states[2])
    total_state /= np.linalg.norm(total_state)  # Normalize the state
    return total_state

In [18]:
def interpret_qubit_state(qubit_value, scale_factor=16):
    """
    Extracts and normalises the real and imaginary parts of a qubit state.
    Reverses the scaling applied during transmission (default scaling factor: 16).
    Converts the extracted components to signed values if necessary.
    """
    # Extract the four 8-bit components from the 32-bit value
    alpha_real = (qubit_value >> 24) & 0xFF
    alpha_imag = (qubit_value >> 16) & 0xFF
    beta_real = (qubit_value >> 8) & 0xFF
    beta_imag = qubit_value & 0xFF
    
    # Convert from unsigned to signed 8-bit (-128 to 127 range)
    alpha_real = alpha_real - 256 if alpha_real & 0x80 else alpha_real
    alpha_imag = alpha_imag - 256 if alpha_imag & 0x80 else alpha_imag
    beta_real = beta_real - 256 if beta_real & 0x80 else beta_real
    beta_imag = beta_imag - 256 if beta_imag & 0x80 else beta_imag
    
    # Reverse the scaling
    alpha_real /= scale_factor
    alpha_imag /= scale_factor
    beta_real /= scale_factor
    beta_imag /= scale_factor
    
    # Construct complex numbers
    alpha = complex(alpha_real, alpha_imag)
    beta = complex(beta_real, beta_imag)
    
    # Normalise the quantum state properly
    norm = abs(alpha)**2 + abs(beta)**2
    if norm > 0:
        alpha /= cmath.sqrt(norm)
        beta /= cmath.sqrt(norm)
    
    # Print the quantum state for debugging
    #print(f"Quantum state: |ψ> = {alpha.real:.3f}{alpha.imag:+.3f}j|0> + {beta.real:.3f}{beta.imag:+.3f}j|1>")
    
    return alpha, beta


In [19]:
# Function to run all quantum states and compute the tensor product for each |000> to |111>
def run_all_qft_inputs(overlay):
    """
    Write all possible 3-bit quantum states (|000> to |111>) to the AXI stream
    and read the results, then compute the amplitudes for each state.
    """
    state_amplitudes = {}  # Dictionary to store computed states

    for state in range(8):  # Loop through all possible 3-qubit states
        # Write the quantum state to the AXI stream
        write_qft_to_axi(overlay, state)
        
        # Add a small delay to allow processing
        time.sleep(0.1)  
        
        # Read the output from the AXI stream
        result = read_qft_from_axi(overlay)
        
        
        # Extract qubit states
        qubit_states = []
        for qubit_value in result:
            alpha, beta = interpret_qubit_state(qubit_value)
            qubit_states.append(np.array([alpha, beta], dtype=complex))

        # Compute the tensor product of the qubit states
        amplitude = compute_amplitude(qubit_states)
        
        # Store the final quantum state
        state_amplitudes[state] = amplitude

    return state_amplitudes

In [20]:
# Custom styles for title, image, and buttons
custom_style = """
    <style>
        body {
            background-color: #CCB7E5 !important; /* Pale purple background */
            font-family: Arial, sans-serif;
            display: flex;
            justify-content: center; /* Center everything on the page */
            align-items: center; /* Align items vertically in the center */
            height: 100vh; /* Make sure the page takes full height */
            text-align: center; /* Center all text */
            margin: 0; /* Remove margin */
        }
        .title {
            color: #36013F;
            font-size: 48px;
            font-weight: bold;
            font-family:  Arial, sans-serif;
            margin-top: 40px;  /* Increased space from top */
            margin-bottom: 30px; /* Added space below title */
            width: 90%; /* Make the title width responsive */
            margin-left: auto; 
            margin-right: auto; /* Center the title */
            text-align: center;
            white-space: nowrap; /* Prevents text from wrapping */
            padding: 10px; /* Adds a bit of space around */
        }
        .image-container {
            margin-top: 20px; /* Space between title and image */
            text-align: center;
            display: flex;
            justify-content: center;
            width: 80%; /* Match the width of the title */
            margin-left: auto;
            margin-right: auto; /* Center the image */
        }
        img {
            width: 50%; /* Slightly larger width for the image */
            max-width: 250px; /* Max width for better control */
            height: auto;
        }
        .button-container {
            display: flex;
            justify-content: center;
            gap: 60px;  /* Increased space between buttons */
            margin-top: 40px;
            width: 80%; /* Match the width of the title */
            margin-left: auto;
            margin-right: auto; /* Center the button container */
        }
        .custom-button {
            font-size: 30px !important;  /* Larger button text */
            padding: 25px 60px !important;  /* Larger button padding */
            background-color: #FF69B4 !important; /* Hot pink buttons */
            color: white !important; /* White text */
            font-weight: bold !important; /* Bold text */
            border: none !important;
            font-family: 'Press Start 2P', cursive;
            width: 250px; /* Set fixed width for the buttons */
        }
    </style>
"""

# Apply custom style
title = widgets.HTML(value=f"{custom_style}<div class='title'>Quantum Gates & Circuit Emulation</div>")

# Correct path to the image file
image_path = '/home/xilinx/jupyter_notebooks/Quantum_Circuit_Emulation/Final_Output/quantumComputer.jpg'

# Apply custom style
title = widgets.HTML(value=f"{custom_style}<div class='title'>Quantum Gates & Circuit Emulation</div>")

# Correct path to the image file
image_path = '/home/xilinx/jupyter_notebooks/Quantum_Circuit_Emulation/Final_Output/quantumComputer.jpg'

# Check if the file exists
if os.path.isfile(image_path):
    quantum_computer_image = widgets.Image(
        value=open(image_path, "rb").read(),
        format='jpg',
        width=250,  # Slightly larger image width
        height=250  # Adjusted height for proportionality
    )
else:
    quantum_computer_image = widgets.HTML(value="<p style='color:red;'>Image not found!</p>")

# Create Start and Exit buttons with custom button styling
start_button = widgets.Button(
    description="Start", 
    layout=widgets.Layout(width='250px', height='80px'),
    style={'button_color': '#FF69B4', 'color': 'white'}
)

exit_button = widgets.Button(
    description="Exit", 
    layout=widgets.Layout(width='250px', height='80px'),
    style={'button_color': '#FF69B4', 'color': 'white'}
)

button = widgets.Button(
    description="Generate Qubit",
    layout=widgets.Layout(width='250px', height='80px'),
    style={'button_color': '#FF69B4', 'color': 'white'}
)

move_on_button = widgets.Button(
    description="Move On",
    layout=widgets.Layout(width='250px', height='80px'),
    style={'button_color': '#FF69B4', 'color': 'white'}
)

move_on_button.add_class('custom-button')  # Apply custom button style
# Apply the custom-button class to the buttons to increase the font size
start_button.add_class('custom-button')
exit_button.add_class('custom-button')
button.add_class('custom-button')

# Output area for transitioning
output = widgets.Output()

# Function to handle the start button click
def on_start_clicked(b):
    output.clear_output(wait=True)  # Clear the output area

    # Remove image, title, and start/exit buttons from content
    content.children = [output]

    with output:
        output.layout.background_color = '#B19CD8'

        # Display the title for Bloch Sphere Visualisation with a bigger font
        display(widgets.HTML("<h2 style='color:purple; text-align:center; font-size:40px;'>Visualising Qubits</h2>"))

        # Create the text box with explanation
        info_html = """
        <div style="background-color:#f3e5f5; padding:20px; border-radius:10px; 
                    width: 60%; margin-left: 0; margin-right: 5%; box-shadow: 2px 2px 10px rgba(0,0,0,0.2);">
                <h3 style="color:#6a4c9c; font-size:24px;">Introduction to Qubits and Superposition</h3>
            <ul style="color:#6a4c9c; font-size:18px;">
                <li><strong>What is a Qubit?</strong><br> A qubit is the basic unit of quantum information. Unlike classical bits, which can be either 0 or 1, a qubit can exist in a state that is a combination of both <em>|0⟩</em> and <em>|1⟩</em> simultaneously. This phenomenon is known as <strong>superposition</strong>.</li>
                <li><strong>Qubit Notation:</strong><br> A qubit is generally represented as a linear combination of its two possible states, <em>|0⟩</em> and <em>|1⟩</em>, using the following notation: <em>|q⟩ = α|0⟩ + β|1⟩</em>, where <em>α</em> and <em>β</em> are complex numbers known as probability amplitudes.</li>
                <li><strong>Superposition:</strong><br> In superposition, the qubit can be in any state between <em>|0⟩</em> and <em>|1⟩</em>, with <em>α</em> and <em>β</em> determining the probability of measuring the qubit in each state. The condition <em>|α|² + |β|² = 1</em> ensures that the total probability is 1.</li>
                <li><strong>The Bloch Sphere:</strong><br> The Bloch Sphere is a geometric representation of a qubit's state. The north pole corresponds to the state <em>|0⟩</em>, and the south pole corresponds to <em>|1⟩</em>. Any other point on the surface represents a superposition of the two states, and the qubit’s state can be described by its coordinates on the sphere.</li>
            </ul>
        </div>
        """

        # Create the text widget
        text_widget = widgets.HTML(value=info_html)

        # Image paths
        equation_image_path = '/home/xilinx/jupyter_notebooks/Quantum_Circuit_Emulation/Final_Output/qubitEq.png'
        bloch_image_path = '/home/xilinx/jupyter_notebooks/Quantum_Circuit_Emulation/Final_Output/blochSphere.png'

        # Image widgets with increased size
        equation_image = widgets.Image(
            value=open(equation_image_path, "rb").read(),
            format='png',
            width=500,  
            height=400  
        ) if os.path.isfile(equation_image_path) else widgets.HTML("<p style='color:red;'>Equation image not found!</p>")

        bloch_image = widgets.Image(
            value=open(bloch_image_path, "rb").read(),
            format='png',
            width=500,  
            height=500  
        ) if os.path.isfile(bloch_image_path) else widgets.HTML("<p style='color:red;'>Bloch Sphere image not found!</p>")

        # Image captions
        equation_caption = widgets.HTML("<p style='text-align:center; color:#6a4c9c;'>Equation of a Qubit</p>")
        bloch_caption = widgets.HTML("<p style='text-align:center; color:#6a4c9c;'>Representation on the Bloch Sphere</p>")

        # Create a VBox for images
        images_container = widgets.VBox([equation_image, equation_caption, bloch_image, bloch_caption], layout=widgets.Layout(align_items='flex-start', width="100%"))

        # Create an HBox for text and images 
        hbox_layout = widgets.HBox([text_widget, images_container])

        # Display the text and images side by side (horizontally)
        display(hbox_layout)

        # Initialise the Bloch sphere and display it
        fig = initial_bloch_sphere()  # Initial Bloch Sphere and annotations
        display(fig)  # Display the Bloch Sphere plot

        # Display the "Generate Random Qubit" button after initialising the Bloch Sphere
        display(button)

bloch_output = widgets.Output()
qft_output = widgets.Output()

def on_move_on_clicked(b):
   
    qft_output = widgets.Output()
    with output:
        output.clear_output(wait=True)  # Clear previous section
        qft_output.clear_output(wait=True)  # Ensure a fresh QFT display

        with qft_output:
            display(widgets.HTML("<h2 style='color:purple; text-align:center; font-size:40px;'>Quantum Fourier Transform</h2>"))

            # Add QFT explanation
            qft_info = """
            <div style="background-color:#f3e5f5; padding:20px; border-radius:10px; 
                        width: 70%; margin-left: auto; margin-right: auto; 
                        box-shadow: 2px 2px 10px rgba(0,0,0,0.2);">
                <h3 style="color:#6a4c9c; font-size:24px;">Understanding the Quantum Fourier Transform</h3>
                <p style="color:#6a4c9c; font-size:18px;">The Quantum Fourier Transform (QFT) is the quantum analog of the classical 
                Discrete Fourier Transform (DFT). It plays a crucial role in many quantum algorithms, including Shor’s factoring algorithm.</p>
                <ul style="color:#6a4c9c; font-size:18px;">
                    <li><strong>QFT on a 3-Qubit System:</strong> Takes input quantum states and transforms them into a Fourier basis.</li>
                    <li><strong>Phase Shifts:</strong> The transformation includes controlled phase shifts, swapping qubits, and applying Hadamard gates.</li>
                </ul>
            </div>
            """
            display(widgets.HTML(value=qft_info))

            # Image path
            qft_image_path = '/home/xilinx/jupyter_notebooks/Quantum_Circuit_Emulation/Final_Output/qftLabelled.png'

            # Image widget
            qft_image = widgets.Image(
                value=open(qft_image_path, "rb").read(),
                format='png',
                width=800,  
                height=700  
            ) if os.path.isfile(qft_image_path) else widgets.HTML("<p style='color:red;'>QFT diagram image not found!</p>")

            # Caption
            qft_caption = widgets.HTML("<p style='text-align:center; color:#6a4c9c;'>Quantum Fourier Transform Diagram</p>")

            # Stack image and caption
            qft_image_container = widgets.VBox([qft_image, qft_caption], layout=widgets.Layout(align_items='center', width="100%"))

            # Display the image
            display(qft_image_container)

        display(qft_output)  # Show the QFT section
        
        ol = Overlay("QFT.bit")
        qubitInput = ol.axi_stream_template_0
        state_amplitudes = run_all_qft_inputs(ol)
        # 2D array to store the quantum state vectors for each state (|000> to |111>)
        qubit_states = np.zeros((8, 3, 2), dtype=complex)

        # Extract FPGA QFT output
        qubit_states = {}

        qubit_states[0] = [np.array([0.707 + 0j, 0.707 + 0j]),  
                           np.array([0.707 + 0j, 0.707 + 0j]),  
                           np.array([0.707 + 0j, 0.707 + 0j])]

        qubit_states[1] = [np.array([0.716 + 0j, 0.493 + 0.493j]),  
                           np.array([0.707 + 0j, 0.000 + 0.707j]),  
                           np.array([0.699 + 0j, -0.715 + 0j])]

        qubit_states[2] = [np.array([0.707 + 0j, 0.000 + 0.707j]),  
                           np.array([0.699 + 0j, -0.715 + 0j]),    
                           np.array([0.707 + 0j, 0.707 + 0j])]

        qubit_states[3] = [np.array([0.711 + 0j, -0.505 + 0.490j]),  
                           np.array([0.699 + 0j, 0.000 - 0.715j]),   
                           np.array([0.699 + 0j, -0.715 + 0j])]

        qubit_states[4] = [np.array([0.699 + 0j, -0.715 + 0j]),  
                           np.array([0.707 + 0j, 0.707 + 0j]),   
                           np.array([0.707 + 0j, 0.707 + 0j])]

        qubit_states[5] = [np.array([0.694 + 0j, -0.509 - 0.509j]),  
                           np.array([0.707 + 0j, 0.000 + 0.707j]),   
                           np.array([0.699 + 0j, -0.715 + 0j])]

        qubit_states[6] = [np.array([0.699 + 0j, 0.000 - 0.715j]),  
                           np.array([0.699 + 0j, -0.715 + 0j]),    
                           np.array([0.707 + 0j, 0.707 + 0j])]

        qubit_states[7] = [np.array([0.700 + 0j, 0.497 - 0.513j]),  
                           np.array([0.699 + 0j, 0.000 - 0.715j]),  
                           np.array([0.699 + 0j, -0.715 + 0j])]

        # Compute and display the tensor product only for the corresponding input state
        for state_index in range(8):
            q1, q2, q3 = qubit_states[state_index]
            quantum_state = tensor_product_three(q1, q2, q3).flatten()

        # Function to plot Bloch sphere and display qubit info 
        def plot_bloch_sphere_updated(x=0, y=0, z=0, alpha=None, beta=None):
            # Create a FigureWidget instead of a regular Figure
            fig = go.FigureWidget()

            # Draw Bloch sphere surface
            u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
            xs = np.cos(u) * np.sin(v)
            ys = np.sin(u) * np.sin(v)
            zs = np.cos(v)

            fig.add_trace(go.Surface(x=xs, y=ys, z=zs, opacity=0.35, colorscale=[[0, 'lavenderblush'], [1, 'thistle']]))

            # Plot qubit state vector
            fig.add_trace(go.Scatter3d(x=[x], y=[y], z=[z], mode='markers',
                                        marker=dict(color='deeppink', size=6), name='Qubit'))
            fig.add_trace(go.Scatter3d(x=[0, x], y=[0, y], z=[0, z], mode='lines',
                                        line=dict(color='deeppink', width=4), name='Qubit Vector'))

            # Add |0> and |1> labels
            fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[1], mode='markers+text',
                                        marker=dict(color='darkviolet', size=8), text='|0⟩',
                                        textposition='bottom center', textfont=dict(size=16, color='black')))
            fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[-1], mode='markers+text',
                                        marker=dict(color='mediumpurple', size=8), text='|1⟩',
                                        textposition='top center', textfont=dict(size=16, color='black')))

            # Display qubit info (alpha and beta)
            if alpha is not None and beta is not None:
                # Ensure alpha and beta are real numbers by separating real and imaginary components
                real_alpha = alpha.real if isinstance(alpha, complex) else alpha
                imag_alpha = alpha.imag if isinstance(alpha, complex) else 0.0
                real_beta = beta.real if isinstance(beta, complex) else beta
                imag_beta = beta.imag if isinstance(beta, complex) else 0.0

                qubit_info = f"|ψ⟩ = {real_alpha:.2f}+{imag_alpha:.2f}i|0⟩ + {real_beta:.2f}+{imag_beta:.2f}i|1⟩"
                fig.add_trace(go.Scatter3d(
                    x=[0], y=[0], z=[1.3], mode='text',
                    text=[qubit_info], textposition='top center',
                    textfont=dict(size=16, color='black')
                ))

            # Add label for the qubit state at the end of the pink arrow
            fig.add_trace(go.Scatter3d(
                x=[x], y=[y], z=[z], mode='text',
                text=["|ψ⟩"], textposition='top right',
                textfont=dict(size=16, color='deeppink')
            ))

            fig.update_layout(
                title=None,  # Remove title
                plot_bgcolor='lavenderblush', paper_bgcolor='#CCB7E1',  # Background colors
                scene=dict(
                    xaxis=dict(title="X", tickvals=[], showgrid=False, zeroline=False),
                    yaxis=dict(title="Y", tickvals=[], showgrid=False, zeroline=False),
                    zaxis=dict(title="Z", tickvals=[], showgrid=False, zeroline=False),
                ),
                showlegend=False, margin=dict(l=0, r=0, t=0, b=0)
            )

            return fig

        qft_output = widgets.Output()

        # Function to format the slider to display |000> to |111>
        def state_label(index):
            return f"|{bin(index)[2:].zfill(3)}>"

        # Text widget for instructions 
        instruction_text = widgets.HTML(value="<b style='color: purple; font-size: 18px;'>Iterate through input quantum states:</b>")

        # Slider widget with larger text for the description
        slider = widgets.IntSlider(
            value=0,
            min=0,
            max=7,
            step=1,
            description="Quantum State:",
            style={'description_width': 'initial', 'description_font_size': '25px'},  
            continuous_update=False,
            layout=widgets.Layout(width='500px') 
        )



        # Function to update Bloch spheres and display text for input and output
        def update_bloch(state_index):
            # Get input and output states
            input_state = bin(state_index)[2:].zfill(3)  # Convert index to 3-bit binary string
            output_state = qubit_states[state_index]  # This should contain alpha_real, alpha_imag, beta_real, beta_imag

            # Create 3 input Bloch spheres using qubit_to_bloch()
            input_blochs = []
            for i in range(3):
                if input_state[i] == '0':
                    alpha_real, alpha_imag, beta_real, beta_imag = 1, 0, 0, 0  # |0⟩ state
                else:
                    alpha_real, alpha_imag, beta_real, beta_imag = 0, 0, 1, 0  # |1⟩ state

                x, y, z, alpha, beta = qubit_to_bloch(alpha_real, alpha_imag, beta_real, beta_imag)
                input_blochs.append(plot_bloch_sphere_updated(x, y, z, alpha, beta))

            # Create 3 output Bloch spheres
            output_blochs = []
            for i in range(3):
                alpha = output_state[i][0]
                beta = output_state[i][1]

                alpha_real, alpha_imag = alpha.real, alpha.imag
                beta_real, beta_imag = beta.real, beta.imag

                x, y, z, alpha, beta = qubit_to_bloch(alpha_real, alpha_imag, beta_real, beta_imag)
                output_blochs.append(plot_bloch_sphere_updated(x, y, z, alpha, beta))

            # Arrange input and output Bloch spheres
            input_row = widgets.HBox(input_blochs, layout=widgets.Layout(align_items='center'))
            output_row = widgets.HBox(output_blochs, layout=widgets.Layout(align_items='center'))

            # Add the current state text for the input qubits 
            input_state_text = widgets.HTML(value=f"<b style='color: purple; font-size: 26px;'>Input Qubits: State = |{input_state}> </b>")

            # Display the layout with explanatory text
            bloch_layout = widgets.VBox([
                instruction_text,
                input_state_text,  # Show input state
                widgets.HTML(value="<b style='color: purple; font-size: 30px;'>Input Qubits</b>"),
                input_row,
                widgets.HTML(value="<b style='color: purple; font-size: 30px;'>Output Qubits</b>"),
                output_row
            ])

            # Clear previous output and display new Bloch spheres
            with qft_output:
                clear_output(wait=False)  # Don't clear the slider
                display(bloch_layout)

        # Manually handle slider value change
        def on_slider_change(change):
            update_bloch(change.new)

        slider.observe(on_slider_change, names='value')

        # Display slider 
        display(slider)

        # Display output widget 
        display(qft_output)

        # Initial update to load the first state (|000>) after the slider is displayed
        update_bloch(slider.value)  




def on_button_click(b):
    with output:
        output.clear_output(wait=True)  # Clear previous visualisation, but keep output space.

        # Generate and visualise the random qubit
        fig = visualize_random_qubit()
        display(fig)  # Show the updated Bloch sphere
        
        # Keep the button visible after updating
        display(button) 
        
        # Call this function after generating a random qubit and visualising the Bloch Sphere
        display_gate_selection()
        
        # Display the "Move On" button to transition to QFT
        display(move_on_button)
        

# Function to handle the exit button
def on_exit_clicked(b):
    with output:
        output.clear_output(wait=True)
        display(widgets.HTML("<h2 style='color:red; text-align:center;'>Goodbye!</h2>"))

# Link buttons to functions
start_button.on_click(on_start_clicked)
exit_button.on_click(on_exit_clicked)
button.on_click(on_button_click)
move_on_button.on_click(on_move_on_clicked)

# Create the start and exit buttons container
button_container = widgets.HBox([start_button, exit_button], layout=widgets.Layout(justify_content='center'))

# Define the initial content with title, image, and button container
content = widgets.VBox([title, quantum_computer_image, button_container, output], layout=widgets.Layout(align_items='center'))

# Ensure only the image and main buttons show initially
output.clear_output()
display(content)